In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import unicodedata
import os


from dotenv import load_dotenv
from utils import espiadinha, get_df
load_dotenv()

True

<h3>Funções para acessar MongoDB Alexandria:  </h3>
*Utilize "help(nome da função)" para entender os atributos da função (exemplo: help(espiadinha))

In [2]:
def from_html_to_text(tag):
    soup = BeautifulSoup(tag, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()   
    text = soup.get_text()
    return text

In [3]:
#credenciais de acesso ao alexandria
server_type = os.getenv("server_type")
MONGO_ALEXANDRIA_USERNAME = os.getenv('MONGO_ALEXANDRIA_USERNAME')
MONGO_ALEXANDRIA_PASSWORD = os.getenv('MONGO_ALEXANDRIA_PASSWORD')
MONGO_ALEXANDRIA_HOST = os.getenv('MONGO_ALEXANDRIA_HOST')
str_conn = f"{server_type}://{MONGO_ALEXANDRIA_USERNAME}:{MONGO_ALEXANDRIA_PASSWORD}@{MONGO_ALEXANDRIA_HOST}" 
client = MongoClient(str_conn)
db = client['alexandria-kroton']

In [4]:
#import das bases do alexandria
cursos={
        'Gastronomia':['PLANEJAMENTO DE CARDÁPIO', 'ADMINISTRAÇÃO APLICADA À GASTRONOMIA'],
        'Engenharia Civil':['FÍSICA GERAL E EXPERIMENTAL - MECÂNICA','QUÍMICA E CIÊNCIA DOS MATERIAIS'],
        'Ciências Contábeis':['ANÁLISE DE CUSTOS', 'MÉTODOS QUANTITATIVOS'],
        'Publicidade e Propaganda':['SISTEMAS BRASILEIROS DE COMUNICAÇÃO', 'SOCIEDADE BRASILEIRA E CIDADANIA']
    }

df_disc=get_df(db,'disciplines',['title','learningUnits','krotonId'])
df_LUs=get_df(db,'learningunits',['_id','classes','krotonId'])
df_cls=get_df(db,'classes',['_id','blocks','krotonId','title'])
df_blk=get_df(db,'blocks',['_id','questions','content','title'])
df_quest=get_df(db,'questions',['_id','type','questionModel','title','description','feedback','alternatives','classId'])

Requisição de conteúdo de aula

In [25]:
sequencia=1
curso_lista=['Gastronomia']
for curso in curso_lista:
    disciplina = cursos[curso][sequencia]
    disciplina = ''.join(c for c in disciplina if not unicodedata.category(c).startswith('C'))
    print(f"Requisitando curso: {curso} - Disciplina: {disciplina}")
    
    #Filtro de Unidades de ensino de uma Disciplina
    filtro_LUs = df_disc[df_disc.title.isin([disciplina])]['learningUnits'].to_list()[-1]

    #Filtro de classes dentro de uma unidade de ensino
    filtro_LU_CLASSES_aninhado = df_LUs[df_LUs._id.isin(filtro_LUs)]['classes'].to_list()

    #Para cada unidade de ensino vamos extrair o conteúdo de cada aula
    dict_conteudo = {}
    for LU_ORDER, filtro_CLASSES_FROM_LU in enumerate(filtro_LU_CLASSES_aninhado):
        LU_ORDER=LU_ORDER+1
        #print(f'Fazendo LU: {LU_ORDER}')
        
        #filtro assets são todos os assets de cada aula de uma unidade de ensino
        filtro_ASSETS = df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)]['blocks'].to_list()
        dict_conteudo[f"Unidade_{LU_ORDER}"] = {}
        #
        for AULA_ORDER, ASSETS_FROM_AULA in enumerate(filtro_ASSETS):
            AULA_ORDER=AULA_ORDER+1
            #print(f'Fazendo aula: {AULA_ORDER}')
            conteudo = f"Unidade_{LU_ORDER} - AULA_{AULA_ORDER} =|> "
            for texto in df_blk[
                (df_blk._id.isin(ASSETS_FROM_AULA))
                &(df_blk.content.notna())
                &(~df_blk.title.isin(['Siga em Frente...','Referências','Saiba mais']))
                ][['title','content']].iterrows():
                conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
            dict_conteudo[f"Unidade_{LU_ORDER}"][f"AULA_{AULA_ORDER}"] = conteudo
    
    

Requisitando curso: Gastronomia - Disciplina: ADMINISTRAÇÃO APLICADA À GASTRONOMIA


C:\Users\renan.vital\AppData\Local\Temp\ipykernel_1096\3182880459.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_1096\3182880459.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_1096\3182880459.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as l

In [6]:
sequencia=1
curso_lista=['Gastronomia']
for curso in curso_lista:
    disciplina = cursos[curso][sequencia]
    disciplina = ''.join(c for c in disciplina if not unicodedata.category(c).startswith('C'))
    print(f"Requisitando curso: {curso} - Disciplina: {disciplina}")
    
    #Filtro de Unidades de ensino de uma Disciplina
    filtro_LUs = df_disc[df_disc.title.isin([disciplina])]['learningUnits'].to_list()[-1]

    #Filtro de classes dentro de uma unidade de ensino
    filtro_LU_CLASSES_aninhado = df_LUs[df_LUs._id.isin(filtro_LUs)]['classes'].to_list()

    #Para cada unidade de ensino vamos extrair o conteúdo de cada aula
    dict_questoes = {}
    for LU_ORDER, filtro_CLASSES_FROM_LU in enumerate(filtro_LU_CLASSES_aninhado):
        LU_ORDER=LU_ORDER+1
        print(f'Fazendo LU: {LU_ORDER}')
        dict_questoes[f"Unidade_{LU_ORDER}"] = {}
        
        #filtro assets são todos os assets de cada aula de uma unidade de ensino
        filtro_ASSETS = df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)]['blocks'].to_list()
        
        for AULA_ORDER, ASSETS_FROM_AULA in enumerate(filtro_ASSETS):

            for id_questions in df_blk[(df_blk._id.isin(ASSETS_FROM_AULA))&(df_blk.title.str.contains('Prova'))]['questions']:
                print(f'Fazendo aula: {AULA_ORDER+1} - {len(id_questions)} questões')
                
                for QUESTION_ORDER, questao_row in enumerate(df_quest[df_quest._id.isin(id_questions)][['title','description','feedback','alternatives']].iterrows()):
                    QUESTION_ORDER=QUESTION_ORDER+1
                    print('Fazendo questão:', QUESTION_ORDER)
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"] = {}
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['title'] = from_html_to_text(questao_row[1]['title'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['description'] = from_html_to_text(questao_row[1]['description'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['feedback'] = from_html_to_text(questao_row[1]['feedback'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['alternatives'] = questao_row[1]['alternatives']
        #break


Requisitando curso: Gastronomia - Disciplina: ADMINISTRAÇÃO APLICADA À GASTRONOMIA
Fazendo LU: 1
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo LU: 2
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo LU: 3
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo LU: 4
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4


In [7]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint='https://openai-dados-lab-poc.openai.azure.com/',
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)   


In [26]:
from langchain_core.documents import Document

for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        questao_texto = dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']+dict_questoes[LU][questao]['feedback']    
        print(LU, questao, questao_texto)
        
        vector_store = FAISS(
            embedding_function=embeddings,
            index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={})
   
        vector_store.add_documents(
            documents=[Document(i) for i in dict_conteudo[LU].values()],
            ids=list(dict_conteudo[LU].keys()))
        
        results = vector_store.similarity_search(questao_texto, k=2)
        dict_questoes[LU][questao]['aula_relacionada_1'] = results[0].page_content
        dict_questoes[LU][questao]['aula_relacionada_2'] = results[1].page_content

Unidade_1 Questao_1 O Poder de Barganha dos Fornecedores é uma das cinco forças no modelo das Cinco Forças de Porter, que ajuda a determinar a atratividade e a competitividade de um mercado ou setor. Esta força específica analisa até que ponto os fornecedores podem influenciar o preço e as condições de seus produtos ou serviços.Considerando o impacto do Poder de Barganha dos Fornecedores na estratégia de negócios, avalie as seguintes asserções e a relação proposta entre elas.I. Empresas podem reduzir o poder de barganha dos fornecedores através da diversificação de fontes de suprimento, estabelecendo relações de longo prazo com diversos fornecedores ou investindo em verticalização. Essas estratégias ajudam a minimizar a dependência de fornecedores específicos, permitindo uma maior flexibilidade operacional e reduzindo o risco de aumento de preços ou diminuição da qualidade dos insumos.PORQUEII. A concentração de fornecedores e a falta de substitutos viáveis são as únicas condições que 

In [34]:
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="https://openai-dados-lab-poc.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment="gpt-4o",
    openai_api_version="2024-02-01"
)

C:\Users\renan.vital\AppData\Local\Temp\ipykernel_1096\3739503124.py:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [39]:
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        questao_texto = dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']+dict_questoes[LU][questao]['feedback']    
        print(LU, questao, questao_texto)
        
        curso_lista[0]
        template_inicial = \
            f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. Your role is develop a
            step by step guide to solve the question below, using two text classes. 
            Question:  {dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']}.
            Correct Answer: {dict_questoes[LU][questao]['feedback']}.
            ----------------------------------
            Context: Use these two texts below to create the guide: 
            {dict_questoes[LU][questao]['aula_relacionada_1'].split('=|>')[0]}: {dict_questoes[LU][questao]['aula_relacionada_1'].split('=|>')[1]}.
            {dict_questoes[LU][questao]['aula_relacionada_2'].split('=|>')[0]}: {dict_questoes[LU][questao]['aula_relacionada_2'].split('=|>')[1]}.
            ----------------------------------
            Action: Briefly explain how to solve the question using the two texts above, indicating the number of the unit and class.
            Important: Just create a list bullet points for each topic from each class on the completion."""

        completion_inicial = llm.invoke(template_inicial)
        dict_questoes[LU][questao]['roteiro_questao'] = completion_inicial.content


Unidade_1 Questao_1 O Poder de Barganha dos Fornecedores é uma das cinco forças no modelo das Cinco Forças de Porter, que ajuda a determinar a atratividade e a competitividade de um mercado ou setor. Esta força específica analisa até que ponto os fornecedores podem influenciar o preço e as condições de seus produtos ou serviços.Considerando o impacto do Poder de Barganha dos Fornecedores na estratégia de negócios, avalie as seguintes asserções e a relação proposta entre elas.I. Empresas podem reduzir o poder de barganha dos fornecedores através da diversificação de fontes de suprimento, estabelecendo relações de longo prazo com diversos fornecedores ou investindo em verticalização. Essas estratégias ajudam a minimizar a dependência de fornecedores específicos, permitindo uma maior flexibilidade operacional e reduzindo o risco de aumento de preços ou diminuição da qualidade dos insumos.PORQUEII. A concentração de fornecedores e a falta de substitutos viáveis são as únicas condições que 

In [48]:
aulas_roteiro=''
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        aulas_roteiro = f"{aulas_roteiro} \n - {dict_questoes[LU][questao]['roteiro_questao']} \n"    

template_inicial = \
    f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. Your role is develop a
    step by step guide to study for an exam. Use the guide provided below: \n {aulas_roteiro}. 
    ----------------------------------
    Action: Create a friendly step by step guide.
    Important:
    - Topics should sorted as this example between triple backticks: ```Unidade 1 - AULA 1, Unidade 1 - AULA 2, Unidade 2 - AULA 1, Unidade 3 - AULA 1```;
    - Completion must contain just the guide;
    - Completion must be in brazilian portuguese;"""

completion_inicial = llm.invoke(template_inicial)
        
completion_inicial.content

'```Unidade 1 - AULA 1```\n- **Análise Estratégica:**\n  - Compreender a importância da análise SWOT para identificar forças, fraquezas, oportunidades e ameaças.\n  - Estudar as cinco forças de Porter para entender a dinâmica competitiva do mercado.\n  - Analisar o ambiente externo e interno que influencia as operações de um negócio gastronômico.\n  - Identificar como fatores macroambientais (econômicos, tecnológicos, sociais, legais, políticos, culturais, demográficos e ecológicos) afetam a estratégia do negócio.\n\n```Unidade 1 - AULA 2```\n- **Planejamento e Diretrizes Organizacionais:**\n  - Compreender o conceito de planejamento como um processo contínuo e sistemático para alcançar objetivos futuros.\n  - Estudar os níveis de planejamento: estratégico (longo prazo), tático (médio prazo) e operacional (curto prazo).\n  - Reconhecer a importância do planejamento na gastronomia para a tomada de decisões informadas e otimização de recursos.\n\n```Unidade 1 - AULA 3```\n- **Estratégias

In [54]:
template_final = \
    f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. You are developing a study guide for your exam.
    This guide must contain:
    - Motivational paragraph: Congratulate the discipline conclusion. Remember how important is this knowledge for his carrer (length: 4 lines). 
    - Instructions:  Tell this guide was build based on meta cognition techniques, and it is a important tool that identifies key subjects on each class that will lickly be asked on the exam. The student should focus and comprehend each topic to perform well.
    - Guide: {completion_inicial.content}
    - Tips paragraph: Remember to sleep well, drink water before the exam, study on a quiet place and to avoid distractions from smartphones or internet.
    - Conclusion: Wish lucky to your student, be kind and positive. 
    ----------------------------------
    Important:
    - Be kind, positive and motivational;
    - Completion must contain the final versio. Therefore, do not need to tell what the paragraf is about;
    - Completion must be in brazilian portuguese;"""
completion_final = llm.invoke(template_final)
        
completion_final.content

'Parabéns por concluir a disciplina de Administração Aplicada à Gastronomia! Esse conhecimento é crucial para sua carreira, pois lhe proporciona as ferramentas necessárias para gerir de forma eficaz e eficiente um negócio gastronômico. A compreensão dos conceitos abordados ao longo do curso permitirá que você tome decisões estratégicas, otimize recursos e alcance o sucesso profissional.\n\nEste guia de estudos foi elaborado com base em técnicas de metacognição, sendo uma ferramenta importante que identifica os principais temas de cada aula, que provavelmente serão abordados no exame. Foque em compreender cada tópico para obter um bom desempenho.\n\n```Unidade 1 - AULA 1```\n- **Análise Estratégica:**\n  - Compreender a importância da análise SWOT para identificar forças, fraquezas, oportunidades e ameaças.\n  - Estudar as cinco forças de Porter para entender a dinâmica competitiva do mercado.\n  - Analisar o ambiente externo e interno que influencia as operações de um negócio gastronôm

In [55]:
print(completion_final.content)

Parabéns por concluir a disciplina de Administração Aplicada à Gastronomia! Esse conhecimento é crucial para sua carreira, pois lhe proporciona as ferramentas necessárias para gerir de forma eficaz e eficiente um negócio gastronômico. A compreensão dos conceitos abordados ao longo do curso permitirá que você tome decisões estratégicas, otimize recursos e alcance o sucesso profissional.

Este guia de estudos foi elaborado com base em técnicas de metacognição, sendo uma ferramenta importante que identifica os principais temas de cada aula, que provavelmente serão abordados no exame. Foque em compreender cada tópico para obter um bom desempenho.

```Unidade 1 - AULA 1```
- **Análise Estratégica:**
  - Compreender a importância da análise SWOT para identificar forças, fraquezas, oportunidades e ameaças.
  - Estudar as cinco forças de Porter para entender a dinâmica competitiva do mercado.
  - Analisar o ambiente externo e interno que influencia as operações de um negócio gastronômico.
  - 